In [1]:
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import os
import shutil
import cv2

In [2]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d huanghanchina/pascal-voc-2012

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/huanghanchina/pascal-voc-2012
License(s): DbCL-1.0
 99% 3.61G/3.63G [00:39<00:00, 161MB/s]
100% 3.63G/3.63G [00:39<00:00, 97.6MB/s]


In [3]:
val_list=['2007_000027.jpg','2007_000032.jpg','2007_000033.jpg','2007_000039.jpg','2007_000042.jpg','2007_000061.jpg',
          '2007_000063.jpg','2007_000068.jpg','2007_000121.jpg','2007_000123.jpg','2007_000129.jpg','2007_000170.jpg',
          '2007_000175.jpg','2007_000187.jpg','2007_000241.jpg','2007_000243.jpg','2007_000250.jpg','2007_000256.jpg',
          '2007_000272.jpg','2007_000323.jpg','2007_000332.jpg','2007_000333.jpg','2007_000346.jpg','2007_000363.jpg',
          '2007_000364.jpg','2007_000392.jpg','2007_000423.jpg','2007_000452.jpg','2007_000464.jpg','2007_000480.jpg',
          '2007_000491.jpg','2007_000504.jpg','2007_000515.jpg','2007_000528.jpg','2007_000529.jpg','2007_000549.jpg',
          '2007_000559.jpg','2007_000572.jpg','2007_000584.jpg','2007_000629.jpg','2007_000636.jpg','2007_000645.jpg',
          '2007_000648.jpg','2007_000661.jpg','2007_000663.jpg','2007_000664.jpg','2007_000676.jpg','2007_000713.jpg',
          '2007_000720.jpg','2007_000727.jpg','2007_000733.jpg','2007_000738.jpg','2007_000762.jpg','2007_000768.jpg',
          '2007_000783.jpg','2007_000793.jpg','2007_000799.jpg','2007_000804.jpg','2007_000807.jpg','2007_000822.jpg',
          '2007_001299.jpg','2007_001311.jpg','2007_001321.jpg','2007_001340.jpg']

In [4]:
!unzip "/content/pascal-voc-2012.zip" -d "/content/dataset/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001829.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001874.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001876.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001882.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001885.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001895.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001896.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001926.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001966.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001971.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001992.png  
  inflating: /content/dataset/voc2012/VOC2012/S

In [5]:
train_images='/content/dataset/VOC2012/JPEGImages/'
train_maps='/content/dataset/VOC2012/Annotations/'

val_images='/content/dataset/VOC2012/ValJPEGImages/'
val_maps='/content/dataset/VOC2012/ValAnnotations/'

classes=['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable',
         'dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

B=2
N_CLASSES=len(classes)
H,W =224,224
SPLIT_SIZE=H//32
print(SPLIT_SIZE)
N_EPOCHS=135
BATCH_SIZE=32

7


In [6]:
!mkdir /content/dataset/VOC2012/ValJPEGImages/
!mkdir /content/dataset/VOC2012/ValAnnotations/

In [7]:
for name in val_list:
  shutil.move(train_maps+name[:-3]+"xml", val_maps+name[:-3]+"xml")

In [8]:
for name in val_list:
  shutil.move(train_images+name, val_images+name)

# Data Preparation

In [9]:
def preprocess_xml(filename):
  tree = ET.parse(filename)
  root = tree.getroot()
  size_tree = root.find('size')
  height = float(size_tree.find('height').text)
  width = float(size_tree.find('width').text)
  bounding_boxes=[]
  for object_tree in root.findall('object'):
    for bounding_box in object_tree.iter('bndbox'):
      xmin = (float(bounding_box.find('xmin').text))
      ymin = (float(bounding_box.find('ymin').text))
      xmax = (float(bounding_box.find('xmax').text))
      ymax = (float(bounding_box.find('ymax').text))
      break
    class_name = object_tree.find('name').text
    class_dict={classes[i]:i for i in range(len(classes))}
    bounding_box = [
        (xmin+xmax)/(2*width),(ymin+ymax)/(2*height),(xmax-xmin)/width,
        (ymax-ymin)/height,class_dict[class_name]]
    bounding_boxes.append(bounding_box)
  return tf.convert_to_tensor(bounding_boxes)

In [10]:
preprocess_xml(val_maps+"2007_000032.xml")

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[ 0.479     ,  0.4644128 ,  0.542     ,  0.37366548,  0.        ],
       [ 0.33      ,  0.37544483,  0.128     ,  0.12455516,  0.        ],
       [ 0.408     ,  0.727758  ,  0.036     ,  0.17437722, 14.        ],
       [ 0.07      ,  0.7597865 ,  0.036     ,  0.17437722, 14.        ]],
      dtype=float32)>

In [11]:
def generate_output(bounding_boxes):
  output_label=np.zeros((SPLIT_SIZE,SPLIT_SIZE,N_CLASSES+5))
  for b in range(len(bounding_boxes)):
    grid_x=bounding_boxes[...,b,0]*SPLIT_SIZE
    grid_y=bounding_boxes[...,b,1]*SPLIT_SIZE
    i=int(grid_x)
    j=int(grid_y)

    output_label[i,j,0:5]=[1.,grid_x%1,grid_y%1,bounding_boxes[...,b,2],bounding_boxes[...,b,3]]
    output_label[i,j,5+int(bounding_boxes[...,b,4])]=1.

  return tf.convert_to_tensor(output_label,tf.float32)

In [12]:
generate_output(preprocess_xml(val_maps+"2007_000032.xml"),)[3][3]

<tf.Tensor: shape=(25,), dtype=float32, numpy=
array([1.        , 0.35299993, 0.25088978, 0.542     , 0.37366548,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ],
      dtype=float32)>

In [13]:
im_paths=[]
xml_paths=[]

val_im_paths=[]
val_xml_paths=[]


for i in os.listdir(train_maps):

  im_paths.append(train_images+i[:-3]+'jpg')
  xml_paths.append(train_maps+i)

for i in os.listdir(val_maps):

  val_im_paths.append(val_images+i[:-3]+'jpg')
  val_xml_paths.append(val_maps+i)

print(len(im_paths),len(xml_paths))
print(len(val_im_paths),len(val_xml_paths))

17061 17061
64 64


In [14]:
train_dataset=tf.data.Dataset.from_tensor_slices((im_paths,xml_paths))
val_dataset=tf.data.Dataset.from_tensor_slices((val_im_paths,val_xml_paths))

In [15]:
def get_imbboxes(im_path,xml_path):
  img=tf.io.decode_jpeg(tf.io.read_file(im_path))
  img=tf.cast(tf.image.resize(img, [H,W]),dtype=tf.float32)

  bboxes=tf.numpy_function(func=preprocess_xml, inp=[xml_path], Tout=tf.float32)
  return img,bboxes

In [16]:
train_dataset=train_dataset.map(get_imbboxes)
val_dataset=val_dataset.map(get_imbboxes)

In [17]:
for i,j in train_dataset.skip(2):
  print(i.shape,j)
  break

(224, 224, 3) tf.Tensor(
[[ 0.475       0.3963964   0.47        0.5705706  19.        ]
 [ 0.401       0.9414414   0.518       0.11711711  8.        ]], shape=(2, 5), dtype=float32)


In [18]:
cv2.imwrite('out_1.jpg',np.array(i))

True

In [19]:
import albumentations as A
transforms = A.Compose([
    A.Resize(H,W),
    A.RandomCrop(
         width=np.random.randint(int(0.9*W),W),
         height=np.random.randint(int(0.9*H),H), p=0.5),
    A.RandomScale(scale_limit=0.1, interpolation=cv2.INTER_LANCZOS4,p=0.5),
    A.HorizontalFlip(p=0.5,),
    A.Resize(H,W),

], bbox_params=A.BboxParams(format='yolo', ))

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [20]:
def aug_albument(image,bboxes):
  augmented=transforms(image=image,bboxes=bboxes)
  return [tf.convert_to_tensor(augmented["image"],dtype=tf.float32),
          tf.convert_to_tensor(augmented["bboxes"],dtype=tf.float32)]

In [21]:
def process_data(image,bboxes):
    aug= tf.numpy_function(func=aug_albument, inp=[image,bboxes], Tout=(tf.float32,tf.float32))
    return aug[0],aug[1]

In [22]:
train_dataset=train_dataset.map(process_data)

In [23]:
for i,j in train_dataset.skip(2):
  print(i.shape,j)
  break

(224, 224, 3) tf.Tensor(
[[ 0.475       0.3963964   0.46999997  0.5705706  19.        ]
 [ 0.40099996  0.9414414   0.518       0.11711717  8.        ]], shape=(2, 5), dtype=float32)


In [24]:
cv2.imwrite('out_2.jpg',np.array(i))

True

In [25]:
def preprocess_augment(img,y):
  img = tf.image.random_brightness(img, max_delta=50.)
  img = tf.image.random_saturation(img, lower=0.5, upper=1.5)
  img = tf.image.random_contrast(img, lower=0.5, upper=1.5)
  #img = tf.image.random_hue(img, max_delta=0.5 )
  img = tf.clip_by_value(img, 0, 255)
  labels=tf.numpy_function(func=generate_output, inp=[y], Tout=(tf.float32))
  return img,labels

In [26]:
def preprocess(img,y):
  img = tf.cast(tf.image.resize(img, size=[H, W]), dtype=tf.float32)

  labels=tf.numpy_function(func=generate_output, inp=[y], Tout=(tf.float32))
  return img,labels

In [27]:
train_dataset=train_dataset.map(preprocess_augment)
val_dataset=val_dataset.map(preprocess)

In [28]:
train_dataset=(
  train_dataset.
  batch(BATCH_SIZE).
  prefetch(tf.data.AUTOTUNE)
)

In [29]:
val_dataset=(
  val_dataset.
  batch(BATCH_SIZE).
  prefetch(tf.data.AUTOTUNE)
)

# Model

In [30]:
NUM_FILTERS=512
OUTPUT_DIM=N_CLASSES+5*B

In [31]:
#base_model = tf.keras.applications.resnet50.ResNet50(
base_model=tf.keras.applications.efficientnet.EfficientNetB1(
    weights='imagenet',
    input_shape=(H,W,3),
    include_top=False,
)
base_model.trainable=False

27018416/27018416 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [32]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, LeakyReLU, Flatten, Dense, Dropout, Reshape
from tensorflow.keras.models import Model

# Define the input shape based on the base model's input
input_layer = Input(shape=(H,W,3))

# Base model (assuming it's already defined)
x = base_model(input_layer)

# Convolutional layers with BatchNormalization and LeakyReLU
x = Conv2D(NUM_FILTERS, (3, 3), padding='same', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.1)(x)

x = Conv2D(NUM_FILTERS, (3, 3), padding='same', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.1)(x)

x = Conv2D(NUM_FILTERS, (3, 3), padding='same', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.1)(x)

x = Conv2D(NUM_FILTERS, (3, 3), padding='same', kernel_initializer='he_normal')(x)
x = LeakyReLU(alpha=0.1)(x)

# Flattening the output for fully connected layers
x = Flatten()(x)

# Fully connected layers
x = Dense(NUM_FILTERS, kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.1)(x)

# Dropout layer
x = Dropout(0.5)(x)

# Output layer with sigmoid activation
output_layer = Dense(SPLIT_SIZE * SPLIT_SIZE * OUTPUT_DIM, activation='sigmoid')(x)
output_layer = Reshape((SPLIT_SIZE, SPLIT_SIZE, OUTPUT_DIM))(output_layer)

# Build the model
model = Model(inputs=input_layer, outputs=output_layer)

model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb1 (Functional)          │ (None, 7, 7, 1280)          │       6,575,239 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 7, 7, 512)           │       5,898,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 7, 7, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 7, 7, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 7, 7, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 7, 7, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 7, 7, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 7, 7, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      12,845,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1470)                │         754,110 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 33,161,285 (126.50 MB)

 Trainable params: 26,581,950 (101.40 MB)

 Non-trainable params: 6,579,335 (25.10 MB)

In [33]:
def compute_iou(boxes1, boxes2):
    boxes1_t = tf.stack([boxes1[..., 0] - boxes1[..., 2] / 2.0,
                         boxes1[..., 1] - boxes1[..., 3] / 2.0,
                         boxes1[..., 0] + boxes1[..., 2] / 2.0,
                         boxes1[..., 1] + boxes1[..., 3] / 2.0],
                        axis=-1)

    boxes2_t = tf.stack([boxes2[..., 0] - boxes2[..., 2] / 2.0,
                         boxes2[..., 1] - boxes2[..., 3] / 2.0,
                         boxes2[..., 0] + boxes2[..., 2] / 2.0,
                         boxes2[..., 1] + boxes2[..., 3] / 2.0],
                        axis=-1)
    lu = tf.maximum(boxes1_t[..., :2], boxes2_t[..., :2])
    rd = tf.minimum(boxes1_t[..., 2:], boxes2_t[..., 2:])

    intersection = tf.maximum(0.0, rd - lu)
    inter_square = intersection[..., 0] * intersection[..., 1]

    square1 = boxes1[..., 2] * boxes1[..., 3]
    square2 = boxes2[..., 2] * boxes2[..., 3]

    union_square = tf.maximum(square1 + square2 - inter_square, 1e-10)
    return tf.clip_by_value(inter_square / union_square, 0.0, 1.0)

In [34]:
def difference(x,y):
  return tf.reduce_sum(tf.square(y-x))

In [ ]:
# #y_true=generate_output([[0.210784,0.616422,0.127451,0.232843,2]])
# y_true=generate_output([[0.509804,0.411765,0.107843,0.245098,3],
#                         [0.210784,0.616422,0.127451,0.232843,2]])
# y_true=np.expand_dims(y_true,axis=0)
# y_pred=np.random.normal(size = (1,7,7,N_CLASSES+5*B))

# y_pred[0][1][4] = [0.9,0.47,0.31,0.12,0.23,   1.0,0.2,0.6,0.1,0.95,    0.9,0.8,0.2,0.6,0.1,0.5,0.9,0.35]
# y_pred[0][3][2] = [0.3,0.01,0.08,0.11,0.54,   0.98,0.56,0.88,0.1,0.24, 0.09,0.018,0.22,0.16,0.01,0.05,0.99,0.3]

# yolo_loss(y_true,y_pred)

In [35]:
def yolo_loss(y_true, y_pred, SPLIT_SIZE, H, W, B):
    # Extract target and predictions
    target = y_true[..., 0]  # Assuming first channel is the object presence indicator
    y_pred_extract = tf.gather_nd(y_pred, tf.where(target == 1))
    y_true_extract = tf.gather_nd(y_true, tf.where(target == 1))

    num_samples = tf.shape(y_pred_extract)[0]

    # Create a mask for when there are no objects in the batch
    no_objects_mask = tf.equal(num_samples, 0)

    # Use tf.cond to handle the case when there are no objects
    loss = tf.cond(no_objects_mask,
                   lambda: tf.constant(0.0),  # No objects in this batch, return zero loss
                   lambda: compute_loss(y_pred, y_true, y_pred_extract, y_true_extract, num_samples, SPLIT_SIZE, H, W, B))

    return loss

def compute_loss(y_pred, y_true, y_pred_extract, y_true_extract, num_samples, SPLIT_SIZE, H, W, B):
    target = y_true[..., 0]
    rescaler = tf.cast(tf.where(target == 1), dtype=tf.float32) * SPLIT_SIZE
    upscaler_1 = tf.concat([rescaler[:, 1:], tf.zeros([num_samples, 2], dtype=tf.float32)], axis=-1)

    target_upscaler_2 = tf.repeat([[float(SPLIT_SIZE), float(SPLIT_SIZE), H, W]],
                                    repeats=num_samples, axis=0) * tf.cast(y_true_extract[..., 1:5], dtype=tf.float32)
    pred_1_upscaler_2 = tf.repeat([[float(SPLIT_SIZE), float(SPLIT_SIZE), H, W]],
                                    repeats=num_samples, axis=0) * tf.cast(y_pred_extract[..., 1:5], dtype=tf.float32)
    pred_2_upscaler_2 = tf.repeat([[float(SPLIT_SIZE), float(SPLIT_SIZE), H, W]],
                                    repeats=num_samples, axis=0) * tf.cast(y_pred_extract[..., 6:10], dtype=tf.float32)

    target_orig = upscaler_1 + target_upscaler_2
    pred_1_orig = upscaler_1 + pred_1_upscaler_2
    pred_2_orig = upscaler_1 + pred_2_upscaler_2

    # IoU calculations and object masks
    iou_1 = compute_iou(target_orig, pred_1_orig)
    iou_2 = compute_iou(target_orig, pred_2_orig)
    mask = tf.cast(tf.greater(iou_2, iou_1), dtype=tf.int32)

    # Object loss calculations
    y_pred_joined = tf.transpose(tf.concat([tf.expand_dims(y_pred_extract[..., 0], axis=0),
                                             tf.expand_dims(y_pred_extract[..., 5], axis=0)], axis=0))

    obj_pred = tf.gather_nd(y_pred_joined, tf.stack([tf.range(num_samples), mask], axis=-1))
    object_loss = tf.reduce_sum(tf.square(obj_pred - 1.0))  # Should be 1

    # No Object Loss
    y_pred_extract_no_obj = tf.gather_nd(y_pred[..., 0:B * 5], tf.where(target == 0))
    no_object_loss_1 = tf.reduce_sum(tf.square(y_pred_extract_no_obj[..., 0]))  # Should be 0
    no_object_loss_2 = tf.reduce_sum(tf.square(y_pred_extract_no_obj[..., 5]))  # Should be 0

    no_object_loss = no_object_loss_1 + no_object_loss_2

    # Class Loss (Updated)
    y_pred_extract_class = tf.gather_nd(y_pred[..., B*5:], tf.where(target == 1))
    class_extract = tf.gather_nd(y_true[..., 5:], tf.where(target == 1))
    class_loss = tf.reduce_sum(tf.square(y_pred_extract_class - class_extract))

    # Bounding Box Loss
    y_pred_extract_bbox = tf.gather_nd(y_pred[..., 0:B * 5], tf.where(target == 1))
    centre_joined = tf.stack([y_pred_extract_bbox[..., 1:3], y_pred_extract_bbox[..., 6:8]], axis=1)
    centre_pred = tf.gather_nd(centre_joined, tf.stack([tf.range(num_samples), mask], axis=-1))
    centre_target = tf.gather_nd(y_true[..., 1:3], tf.where(target == 1))
    centre_loss = tf.reduce_sum(tf.square(centre_pred - centre_target))

    size_joined = tf.stack([y_pred_extract_bbox[..., 3:5], y_pred_extract_bbox[..., 8:10]], axis=1)
    size_pred = tf.gather_nd(size_joined, tf.stack([tf.range(num_samples), mask], axis=-1))
    size_target = tf.gather_nd(y_true[..., 3:5], tf.where(target == 1))
    size_loss = tf.reduce_sum(tf.square(tf.math.sqrt(tf.abs(size_pred)) - tf.math.sqrt(tf.abs(size_target))))

    box_loss = centre_loss + size_loss

    # Hyperparameters
    lambda_coord = 5.0
    lambda_no_obj = 0.5

    # Final loss computation
    loss = object_loss + (lambda_no_obj * no_object_loss) + (lambda_coord * box_loss) + class_loss
    return loss

In [36]:
checkpoint_filepath='/content/drive/MyDrive/Bang/yolo_efficientnet_b1_new.weights.h5'
callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [37]:
def scheduler(epoch, lr):
  if epoch < 40:
    return 1e-3
  elif epoch>=40 and epoch<80:
    return 5e-4
  else:
    return 1e-4

In [38]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [39]:
model.compile(
    optimizer='adam',
    loss=lambda y_true, y_pred: yolo_loss(y_true, y_pred, SPLIT_SIZE, H, W, B),
    run_eagerly=True
)


In [40]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.load_weights(checkpoint_filepath)

In [41]:
for images, labels in train_dataset.take(1):
    print(images.shape, labels.shape)


(32, 224, 224, 3) (32, 7, 7, 25)


In [42]:
history = model.fit(
  train_dataset,
  validation_data=val_dataset,
  verbose=1,
  epochs=2,
  callbacks = [lr_callback,callback]
)

Epoch 1/2
534/534 ━━━━━━━━━━━━━━━━━━━━ 761s 1s/step - loss: 249.3254 - val_loss: 190.3191 - learning_rate: 0.0010
Epoch 2/2
534/534 ━━━━━━━━━━━━━━━━━━━━ 741s 1s/step - loss: 156.3196 - val_loss: 173.7344 - learning_rate: 0.0010


# Testing

In [43]:
model.load_weights(checkpoint_filepath)

In [44]:
!mkdir outputs/

In [48]:
COCO_PATH='/content/drive/MyDrive/Bang/coco_images/'

In [49]:
def model_test(filename):
  try:
    test_path=COCO_PATH+filename

    print(test_path)

    img=cv2.resize(cv2.imread(test_path),(H,W))

    image=tf.io.decode_jpeg(tf.io.read_file(test_path))
    image=tf.image.resize(image, [H,W])

    output=model.predict(np.expand_dims(image, axis = 0))

    THRESH=.25

    object_positions=tf.concat(
        [tf.where(output[...,0]>=THRESH),tf.where(output[...,5]>=THRESH)],axis=0)
    print(object_positions)
    selected_output=tf.gather_nd(output,object_positions)
    print(selected_output)
    final_boxes=[]
    final_scores=[]

    for i,pos in enumerate(object_positions):
      for j in range(2):
        if selected_output[i][j*5]>THRESH:
          output_box=tf.cast(output[pos[0]][pos[1]][pos[2]][(j*5)+1:(j*5)+5],dtype=tf.float32)

          x_centre=(tf.cast(pos[1],dtype=tf.float32)+output_box[0])*32
          y_centre=(tf.cast(pos[2],dtype=tf.float32)+output_box[1])*32

          x_width,y_height=tf.math.abs(H*output_box[2]),tf.math.abs(W*output_box[3])

          x_min,y_min=int(x_centre-(x_width/2)),int(y_centre-(y_height/2))
          x_max,y_max=int(x_centre+(x_width/2)),int(y_centre+(y_height/2))

          if(x_min<=0):x_min=0
          if(y_min<=0):y_min=0
          if(x_max>=W):x_max=W
          if(y_max>=H):y_max=H
          final_boxes.append(
              [x_min,y_min,x_max,y_max,
              str(classes[tf.argmax(selected_output[...,10:],axis=-1)[i]])])
          final_scores.append(selected_output[i][j*5])
    print(final_scores)
    print('finalboxes',final_boxes)
    final_boxes=np.array(final_boxes)

    object_classes=final_boxes[...,4]
    nms_boxes=final_boxes[...,0:4]

    nms_output=tf.image.non_max_suppression(
        nms_boxes,final_scores,max_output_size=100,iou_threshold=0.2,
        score_threshold=float('-inf')
    )
    print(nms_output)

    for i in nms_output:
      cv2.rectangle(
          img,
          (int(final_boxes[i][0]),int(final_boxes[i][1])),
          (int(final_boxes[i][2]),int(final_boxes[i][3])),(0,0,255),1)
      cv2.putText(
          img,
          final_boxes[i][-1],
          (int(final_boxes[i][0]),int(final_boxes[i][1])+15),
          cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(2,225,155),1
          )

    cv2.imwrite('/content/outputs/'+filename[:-4]+'_det'+'.jpg',cv2.resize(img,(384,384)))
  except:
    print("NO object found !!!")

In [50]:
for filename in os.listdir(COCO_PATH):
  model_test(filename)

/content/drive/MyDrive/Bang/coco_images/2007_001321.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 676ms/step
tf.Tensor(
[[0 3 3]
 [0 2 3]
 [0 3 4]], shape=(3, 3), dtype=int64)
tf.Tensor(
[[0.5070571  0.36519322 0.8122189  0.89596176 0.6135364  0.07273909
  0.5880629  0.5685128  0.3520704  0.31163523 0.02883933 0.01007863
  0.01350926 0.00513289 0.00681379 0.00396489 0.790216   0.07869826
  0.01255293 0.0329945  0.01540485 0.08011366 0.01689729 0.03079392
  0.01000633 0.00701903 0.01208905 0.02177459 0.01410051 0.00590952]
 [0.03537805 0.60323477 0.2766131  0.38089308 0.36071035 0.29027477
  0.5284607  0.7536946  0.7904839  0.7400959  0.05375865 0.01335814
  0.03165173 0.02863855 0.01007746 0.0795941  0.2766265  0.0794088
  0.01622192 0.05760697 0.01735592 0.15642188 0.04549766 0.03606917
  0.0605524  0.00647013 0.03428356 0.00969613 0.03237    0.01220173]
 [0.02836279 0.70521194 0.18346953 0.2984578  0.45994765 0.48888806
  0.4591184  0.23120308 0.9663217  0.73055005 0.03846958 0.02445342
  0.034015